# Generator's LAB

Podstawą wszystkich rozwiązań typu RAG (ang. Retrieval Augmented Generation) jest komponent wyszukiwania. Wyszukiwanie polega na identyfikacji odpowiednich dokumentów, w naszym RAGOwym przypadku takich dokumentów, które potencjalnie zawierają informacje kluczowe do odpowiedzenia na postawione pytanie. Definicja dokumentu jest też rzeczą płynną, raz to może być paragraf, a raz strona, czy rozdział, zależy to od zadania, oraz modeli jakie używamy.

Po realizacji procesu wyszukania top N dokumentów (np. N=100), bardzo często dokonuje się re-ranking aby wybrać top5-10 najlepszych i takie wyselekcjonowane dokumenty tworzą tzw. kontekst. Kontekst to konkatenecja tych top5-10 najlepszych dokumentów. Mówiąc konkatentacja mam na myśli wylistowanie tych dokumentów w formie jednego tekstu.

Na wykładzie starałem się wyraźnie zwrócić uwagę na to, że taki kontekst staje się kluczowym elementem prompta wysyłanego do modelu LLM pełniącego rolę generatora. Generator to LLM pełniący rolę ostatniegu komponentu w RAG architekturze oraz produkującego finalną odpowiedź w języku naturalnym. Generator to model LLM, który po otrzymaniu na wejściu, w prompcie, pytanie i kontekst generuje odpowiedź w języku naturalnym na zadane pytanie w dostarczonym ujętym kontekście informacyjnym. Reasumując, prompt do generatora zawiera pytanie i kontekst, a completion to odpowiedź generatora.

Przykładowey szablon prompta do generatora, który posiada listę dokumentów budujących kontekst informacyjny, oraz instrukcje, a na końcu włąsciwe pytanie.


```
Numerowana lista dokumentów jest poniżej:
---------------------
<results>{% for doc in docs %}
Dokument: {{ loop.index0 }}
{{ doc }}
{% endfor %}</results>
---------------------
Odpowiedz na pytanie użytkownika wykorzystując tylko informacje znajdujące się w dokumentach, a nie wcześniejszą wiedzę.
Udziel wysokiej jakości, poprawnej gramatycznie odpowiedzi w języku polskim. Odpowiedź powinna zawierać cytowania do dokumentów, z których pochodzą informacje. Zacytuj dokument za pomocą symbolu [nr_dokumentu] powołując się na fragment np. [0] dla fragmentu z dokumentu 0. Jeżeli w dokumentach nie ma informacji potrzebnych do odpowiedzi na pytanie, zamiast odpowiedzi zwróć tekst: "Nie udało mi się odnaleźć odpowiedzi na pytanie".

Pytanie: {{ question }}

```


Do pracy wymagamy zainstalowania pakietów - `transformers` oraz `datasets`


In [1]:
!pip install -U transformers datasets
!pip install bitsandbytes
!pip install flash-attn --no-build-isolation

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 14.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 

Zanim przejdziemy do zdefiniowania szczegółowych zadań sprawdźmy jak użyć powyższy szablon prompta RAGowego do istniejącego LLMa, który ma kompetencje bycia generatorem w RAGach.


Przykładowe użycie generatora na statycznych danych z wykorzystaniem pipeline z transformers oraz modeli PLLuM LLama 8b

In [ ]:


model_name = "CYFRAGOVPL/Llama-PLLuM-8B-instruct"

# Prompt do generatora z kontekstem opartym na przykładowych wyszukanych tekstach
prepared_prompt = """
Numerowana lista dokumentów jest poniżej:
---------------------

Dokument: 0

Uzyskaj paszport - usługa dla osoby dorosłej
Co musisz przygotować
twoje jedno aktualne, kolorowe zdjęcie - zrób je u fotografa nie wcześniej niż 6 miesięcy przed złożeniem wniosku. Jeśli chcesz je zrobić samodzielnie - sprawdź, jak wykonać zdjęcie do paszportu,
dowód opłaty za paszport (po złożeniu wniosku opłata nie podlega zwrotowi),
jeśli przysługuje ci prawo do zniżki lub zwolnienia z opłaty - dokument, który to potwierdzi, na przykład ważna legitymacja studencka (szczegóły znajdziesz w sekcji Zwolnienia i zniżki),
ważny paszport, którego obecnie używasz. Jeśli nie masz ważnego paszportu - weź ze sobą ważny dowód osobisty,
jeśli twoje nazwisko zostało zmienione po ślubie za granicą - skrócony lub zupełny odpis polskiego aktu małżeństwa.


Dokument: 1

Uzyskaj paszport - usługa dla osoby dorosłej
Jak odbierzesz dokument

Osobiście w punkcie paszportowym - tam, gdzie złożysz wniosek.

Weź ze sobą:

ważny dokument tożsamości,
aktualny dokument paszportowy - jeśli masz,
poprzedni paszport - jeśli masz. Zostanie on anulowany.
Przy odbiorze paszportu sprawdź - czy twoje dane osobowe i biometryczne są poprawne.


---------------------
Odpowiedz na pytanie użytkownika wykorzystując tylko informacje znajdujące się w dokumentach, a nie wcześniejszą wiedzę.
Udziel wysokiej jakości, poprawnej gramatycznie odpowiedzi w języku polskim. Odpowiedź powinna zawierać cytowania do dokumentów, z których pochodzą informacje. Zacytuj dokument za pomocą symbolu [nr_dokumentu] powołując się na fragment np. [0] dla fragmentu z dokumentu 0. Jeżeli w dokumentach nie ma informacji potrzebnych do odpowiedzi na pytanie, zamiast odpowiedzi zwróć tekst: "Nie udało mi się odnaleźć odpowiedzi na pytanie".

Pytanie: Jak odebrać paszport?"""

import transformers
import torch


pipeline = transformers.pipeline(
    "text-generation",
    model=model_name,
    model_kwargs={"torch_dtype": torch.float16}, #float16 if not ampere/hopper gpu
    device_map="auto",
)

messages = [
    {"role": "system", "content": "You are an administration chatbot assistant!"},
    {"role": "user", "content": prepared_prompt},
]

outputs = pipeline(
    messages,
    max_new_tokens=256,
)
print(outputs[0]["generated_text"][-1])




/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/846 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/52.1k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/525 [00:00<?, ?B/s]

Device set to use cuda:0


{'role': 'assistant', 'content': 'Aby odebrać paszport, należy osobiście stawić się w punkcie paszportowym, w którym złożono wniosek [1]. Przy odbiorze paszportu należy mieć przy sobie ważny dokument tożsamości, a także aktualny dokument paszportowy, jeśli posiada się taki dokument [1]. Ponadto, przy odbiorze paszportu, należy mieć przy sobie poprzedni paszport, który zostanie anulowany [1]. Przy odbiorze paszportu należy sprawdzić, czy dane osobowe i biometryczne są poprawne [1].'}


Powyżej pokazałem jak użyć modelu PLLuM LLama 8b Instruct (CYFRAGOVPL/Llama-PLLuM-8B-instruct) do generacji na bazie prompta zawierającego kontekst (lista dokumentów) i pytanie. Teraz czas na twoje zadania.

ZADANIE #1 - Sprawdź jak działa kod wyżej umieszczony na modelu typu chat - CYFRAGOVPL/Llama-PLLuM-8B-chat, czy widzisz różnice, oraz opisz co zaobserwowałeś jak device_map zmienisz z "auto" na "cuda", wytłumacz dlaczego tak się dzieje?

## **Odpowiedź:**

Wygenerowany przez model -chat tekst okazał się trochę krótszy od tekstu wygenerowanego przez model -instruct. Semantycznie jednak zdania były identyczne - te same informacje zostały przekazane w tej samej kolejności w obu odpowiedziach. \\

**Teksty wygenerowany przez model -instruct:** \\

Device set to use cuda:0 \\
{'role': 'assistant', 'content': 'Aby odebrać paszport, należy osobiście udać się do punktu paszportowego, w którym złożono wniosek [1]. Przy odbiorze paszportu należy posiadać ważny dokument tożsamości, a także aktualny dokument paszportowy, jeśli się go posiada [1]. Ponadto, należy mieć przy sobie poprzedni paszport, który zostanie anulowany [1]. Przy odbiorze paszportu należy sprawdzić, czy dane osobowe i biometryczne są poprawne [1].'}

**Tekst wygenerowany przez model -chat:** \\

Device set to use cuda:0 \\
{'role': 'assistant', 'content': 'Aby odebrać paszport, należy osobiście udać się do punktu paszportowego, w którym złożono wniosek [1]. Przy odbiorze paszportu należy mieć przy sobie ważny dokument tożsamości, aktualny dokument paszportowy (jeśli jest) oraz poprzedni paszport (jeśli jest), który zostanie anulowany [1]. Przy odbiorze należy również sprawdzić, czy dane osobowe i biometryczne są poprawne [1].'}


## **Zmiana device_map na cuda**

W eksperymencie uruchomiłam model CYFRAGOVPL/Llama-PLLuM-8B-chat z dwoma ustawieniami "auto" oraz "cuda". \\

**device_map="auto"** \\
Model załadował się poprawnie – mimo że karta graficzna miała ograniczoną ilość pamięci. Dzięki opcji "auto" model automatycznie rozdzielił obliczenia między GPU i CPU, co pozwoliło go uruchomić, nawet jeśli nie cały model zmieścił się na karcie graficznej. Generacja odpowiedzi działała, chociaż była nieco wolniejsza, bo część operacji była wykonywana przez CPU.

**device_map="cuda"** \\
Po ustawieniu "cuda" otrzymałam błąd:

OutOfMemoryError: CUDA out of memory. Tried to allocate 1004.00 MiB... \\

Oznacza to, że model próbował załadować się w całości na GPU, ale nie zmieścił się w dostępnej pamięci. W efekcie generowanie nie ruszyło w ogóle.

In [ ]:


model_name = "CYFRAGOVPL/Llama-PLLuM-8B-chat"

# Prompt do generatora z kontekstem opartym na przykładowych wyszukanych tekstach
prepared_prompt = """
Numerowana lista dokumentów jest poniżej:
---------------------

Dokument: 0

Uzyskaj paszport - usługa dla osoby dorosłej
Co musisz przygotować
twoje jedno aktualne, kolorowe zdjęcie - zrób je u fotografa nie wcześniej niż 6 miesięcy przed złożeniem wniosku. Jeśli chcesz je zrobić samodzielnie - sprawdź, jak wykonać zdjęcie do paszportu,
dowód opłaty za paszport (po złożeniu wniosku opłata nie podlega zwrotowi),
jeśli przysługuje ci prawo do zniżki lub zwolnienia z opłaty - dokument, który to potwierdzi, na przykład ważna legitymacja studencka (szczegóły znajdziesz w sekcji Zwolnienia i zniżki),
ważny paszport, którego obecnie używasz. Jeśli nie masz ważnego paszportu - weź ze sobą ważny dowód osobisty,
jeśli twoje nazwisko zostało zmienione po ślubie za granicą - skrócony lub zupełny odpis polskiego aktu małżeństwa.


Dokument: 1

Uzyskaj paszport - usługa dla osoby dorosłej
Jak odbierzesz dokument

Osobiście w punkcie paszportowym - tam, gdzie złożysz wniosek.

Weź ze sobą:

ważny dokument tożsamości,
aktualny dokument paszportowy - jeśli masz,
poprzedni paszport - jeśli masz. Zostanie on anulowany.
Przy odbiorze paszportu sprawdź - czy twoje dane osobowe i biometryczne są poprawne.


---------------------
Odpowiedz na pytanie użytkownika wykorzystując tylko informacje znajdujące się w dokumentach, a nie wcześniejszą wiedzę.
Udziel wysokiej jakości, poprawnej gramatycznie odpowiedzi w języku polskim. Odpowiedź powinna zawierać cytowania do dokumentów, z których pochodzą informacje. Zacytuj dokument za pomocą symbolu [nr_dokumentu] powołując się na fragment np. [0] dla fragmentu z dokumentu 0. Jeżeli w dokumentach nie ma informacji potrzebnych do odpowiedzi na pytanie, zamiast odpowiedzi zwróć tekst: "Nie udało mi się odnaleźć odpowiedzi na pytanie".

Pytanie: Jak odebrać paszport?"""

import transformers
import torch

pipeline = transformers.pipeline(
    "text-generation",
    model=model_name,
    model_kwargs={"torch_dtype": torch.float16}, #float16 if not ampere/hopper gpu
    device_map="auto",
)

messages = [
    {"role": "system", "content": "You are an administration chatbot assistant!"},
    {"role": "user", "content": prepared_prompt},
]

outputs = pipeline(
    messages,
    max_new_tokens=256,
)
print(outputs[0]["generated_text"][-1])




/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/931 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/52.1k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/525 [00:00<?, ?B/s]

Device set to use cuda:0


{'role': 'assistant', 'content': 'Aby odebrać paszport, należy osobiście stawić się w punkcie paszportowym, w którym złożono wniosek [1]. Przy odbiorze paszportu należy mieć przy sobie ważny dokument tożsamości, a także aktualny dokument paszportowy oraz poprzedni paszport, który zostanie anulowany [1]. Przy odbiorze należy również sprawdzić, czy dane osobowe i biometryczne są poprawne [1].'}


ZADANIE #2 - Przepisz wyżej pokazany kod tak aby nie używać pipelinu, tylko natywnego podejścia transformers z użyciem takich komend jak AutoModelForCausalLM.from_pretrained(), AutoTokenizer.from_pretrained() i tokenizer.apply_chat_template(), oraz model.generate()

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_name = "CYFRAGOVPL/Llama-PLLuM-8B-instruct"

# Przygotowany prompt z dokumentami i pytaniem
prepared_prompt = """
Numerowana lista dokumentów jest poniżej:
---------------------

Dokument: 0

Uzyskaj paszport - usługa dla osoby dorosłej
Co musisz przygotować
twoje jedno aktualne, kolorowe zdjęcie - zrób je u fotografa nie wcześniej niż 6 miesięcy przed złożeniem wniosku. Jeśli chcesz je zrobić samodzielnie - sprawdź, jak wykonać zdjęcie do paszportu,
dowód opłaty za paszport (po złożeniu wniosku opłata nie podlega zwrotowi),
jeśli przysługuje ci prawo do zniżki lub zwolnienia z opłaty - dokument, który to potwierdzi, na przykład ważna legitymacja studencka (szczegóły znajdziesz w sekcji Zwolnienia i zniżki),
ważny paszport, którego obecnie używasz. Jeśli nie masz ważnego paszportu - weź ze sobą ważny dowód osobisty,
jeśli twoje nazwisko zostało zmienione po ślubie za granicą - skrócony lub zupełny odpis polskiego aktu małżeństwa.

Dokument: 1

Uzyskaj paszport - usługa dla osoby dorosłej
Jak odbierzesz dokument

Osobiście w punkcie paszportowym - tam, gdzie złożysz wniosek.

Weź ze sobą:

ważny dokument tożsamości,
aktualny dokument paszportowy - jeśli masz,
poprzedni paszport - jeśli masz. Zostanie on anulowany.
Przy odbiorze paszportu sprawdź - czy twoje dane osobowe i biometryczne są poprawne.

---------------------
Odpowiedz na pytanie użytkownika wykorzystując tylko informacje znajdujące się w dokumentach, a nie wcześniejszą wiedzę.
Udziel wysokiej jakości, poprawnej gramatycznie odpowiedzi w języku polskim. Odpowiedź powinna zawierać cytowania do dokumentów, z których pochodzą informacje. Zacytuj dokument za pomocą symbolu [nr_dokumentu] powołując się na fragment np. [0] dla fragmentu z dokumentu 0. Jeżeli w dokumentach nie ma informacji potrzebnych do odpowiedzi na pytanie, zamiast odpowiedzi zwróć tekst: "Nie udało mi się odnaleźć odpowiedzi na pytanie".

Pytanie: Jak odebrać paszport?
"""
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
)

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.float16,
    quantization_config=bnb_config
)

messages = [
    {"role": "system", "content": "You are an administration chatbot assistant!"},
    {"role": "user", "content": prepared_prompt}
]

formatted_prompt = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)

inputs = tokenizer(formatted_prompt, return_tensors="pt").to(model.device)

outputs = model.generate(
    **inputs,
    max_new_tokens=256,
    do_sample=False,
    temperature=0.7
)

response = tokenizer.decode(outputs[0], skip_special_tokens=True)

answer = response[len(formatted_prompt):].strip()

print(answer)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


sobiście udać się do punktu paszportowego, w którym złożono wniosek, [1]. Przy odbiorze paszportu należy mieć przy sobie ważny dokument tożsamości oraz aktualny dokument paszportowy, jeśli jest się w jego posiadaniu, a także poprzedni paszport, który zostanie anulowany [1]. Przy odbiorze paszportu należy sprawdzić, czy dane osobowe i biometryczne są poprawne [1].


ZADANIE #3 - Do kodu napisanego w zadaniu 2 wprowadź optymalizacje obliczeń atencji np. flash-attention-2, aby zmniejszyć zużycie pamięci i szybkość generacji, opisz co zaobserwowałeś, czy coś się zmieniło i dlaczego nie, albo tak?

W celu przyspieszenia generowania i zmniejszenia zużycia pamięci, podjęłam próbę zastosowania mechanizmu flash_attention_2 poprzez ustawienie parametru: attn_implementation="flash_attention_2".

Jednak po uruchomieniu kodu pojawił się błąd: \\
**NameError: name '_flash_supports_window_size' is not defined**

Ten błąd oznacza, że model lub aktualna wersja biblioteki transformers i/lub flash-attn nie obsługuje flash_attention_2 dla tego konkretnego modelu lub architektury. \\
Nie udało mi się wykorzystać flash_attention_2 w tym przypadku ze względu na brak wsparcia w modelu lub konflikty w wersjach bibliotek. Generacja działa poprawnie dopiero po użyciu implementacji attn_implementation="eager", jednak jest ona mniej wydajna — czas generacji wynosił około 4 minut.

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_name = "CYFRAGOVPL/Llama-PLLuM-8B-instruct"

# Przygotowany prompt z dokumentami i pytaniem
prepared_prompt = """
Numerowana lista dokumentów jest poniżej:
---------------------

Dokument: 0

Uzyskaj paszport - usługa dla osoby dorosłej
Co musisz przygotować
twoje jedno aktualne, kolorowe zdjęcie - zrób je u fotografa nie wcześniej niż 6 miesięcy przed złożeniem wniosku. Jeśli chcesz je zrobić samodzielnie - sprawdź, jak wykonać zdjęcie do paszportu,
dowód opłaty za paszport (po złożeniu wniosku opłata nie podlega zwrotowi),
jeśli przysługuje ci prawo do zniżki lub zwolnienia z opłaty - dokument, który to potwierdzi, na przykład ważna legitymacja studencka (szczegóły znajdziesz w sekcji Zwolnienia i zniżki),
ważny paszport, którego obecnie używasz. Jeśli nie masz ważnego paszportu - weź ze sobą ważny dowód osobisty,
jeśli twoje nazwisko zostało zmienione po ślubie za granicą - skrócony lub zupełny odpis polskiego aktu małżeństwa.

Dokument: 1

Uzyskaj paszport - usługa dla osoby dorosłej
Jak odbierzesz dokument

Osobiście w punkcie paszportowym - tam, gdzie złożysz wniosek.

Weź ze sobą:

ważny dokument tożsamości,
aktualny dokument paszportowy - jeśli masz,
poprzedni paszport - jeśli masz. Zostanie on anulowany.
Przy odbiorze paszportu sprawdź - czy twoje dane osobowe i biometryczne są poprawne.

---------------------
Odpowiedz na pytanie użytkownika wykorzystując tylko informacje znajdujące się w dokumentach, a nie wcześniejszą wiedzę.
Udziel wysokiej jakości, poprawnej gramatycznie odpowiedzi w języku polskim. Odpowiedź powinna zawierać cytowania do dokumentów, z których pochodzą informacje. Zacytuj dokument za pomocą symbolu [nr_dokumentu] powołując się na fragment np. [0] dla fragmentu z dokumentu 0. Jeżeli w dokumentach nie ma informacji potrzebnych do odpowiedzi na pytanie, zamiast odpowiedzi zwróć tekst: "Nie udało mi się odnaleźć odpowiedzi na pytanie".

Pytanie: Jak odebrać paszport?
"""

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
)

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.float16,
    attn_implementation="eager",
    quantization_config=bnb_config
)

messages = [
    {"role": "system", "content": "You are an administration chatbot assistant!"},
    {"role": "user", "content": prepared_prompt}
]

formatted_prompt = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)

inputs = tokenizer(formatted_prompt, return_tensors="pt").to(model.device)

outputs = model.generate(
    **inputs,
    max_new_tokens=256,
    do_sample=False,
    temperature=0.7
)

response = tokenizer.decode(outputs[0], skip_special_tokens=True)

answer = response[len(formatted_prompt):].strip()

print(answer)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/52.1k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/525 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/846 [00:00<?, ?B/s]

...

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


sobiście udać się do punktu paszportowego, w którym złożono wniosek, [1]. Przy odbiorze paszportu należy mieć przy sobie ważny dokument tożsamości oraz aktualny dokument paszportowy, jeśli jest się w jego posiadaniu, a także poprzedni paszport, który zostanie anulowany [1]. Przy odbiorze paszportu należy sprawdzić, czy dane osobowe i biometryczne są poprawne [1].
